# The 2-variable polynomial invariant of knotted 4-valent graphs

This notebook is intended to show the process of finding the multiset of 2-variable polynomial invariants for a choice of knotted 4-valent graph $G$, and a knot invariant polynomial. It uses Python code from the package Graphpoly to do this; here are the steps used, given a specific Dowker-Thistletwaite sequence for a knotted graph.

1. Find the orientation of the graph nodes by running the DT algorithm on the sequence, giving the $f(i)$ function on node labels $i$. The convention is that $f(i) = +1$ if the other arc crosses from right to left, and $f(i) = -1$ if the other arc crosses from left to right.
1. Find the planar diagram (PD) code for the graph, using the DT sequence and the orientation just found.
1. From the PD code, find all possible Eulerian circuits through the graph. This gives the multiset of induced knot diagrams for the knot.
1. Choose a link polynomial $L(K)$ which is invariant for any diagram of the link $K$. Use the skein relation given below for the vertices of the graph to find a polynomial in the variables $c, s$, with coefficients given by link diagrams. The brackets with subscript '$L$' refer to evaluating the resulting link using the link polynomial $L(K)$.

![](./img/eqns.png)

For this notebook, we will use the example of the graph given by the DT sequence $3^\ell 5^u 7^- 1^+$. This is represented as a Python list by

```python
    seq = [[0, 3, -2], [2, 5, 2], [4, 7, -1], [6, 1, 1]]
```

For each node, there is a list of three numbers in this list. The first element is an even number, the second an odd; these are the two labels for that particular node. The node labels are in the set $\{0, 1, \cdots, 2N - 1\}$, for a graph with $N$ nodes. The third element gives the crossing or vertex type: -2 for a vertex with the upper edge on the odd label (label superscript $\ell$), -1 for a crossing with the upper edge on the odd label (label superscript $-$), and the positive numbers are the flipped cases (superscripts $u$ and $+$).

## Find the orientation of the graph nodes

First, we import the function `isRealizable(seq, f_list = False)`. If the function is given a Python list `seq`, the function returns `True` if the sequence is a realizable DT sequence, and `False` if the sequence does not represent a planar graph. Note that the given sequence `seq` may or may not include the crossing or vertex type. If the parameter `f_list` is set to `True`, for a realizable sequence, the function will return a list of $f(i)$ values for each node label $i$. These indicate the crossing information at the node $i$ for the Eulerian circuit described by the DT sequence. See "Classification of knot projections", Topology Appl. 16, 19–31 (1983), for details.

In [1]:
from Graphpoly.isRealizable import isRealizable

The next command creates the orientation list `orientList` for the example DT sequence `seq`. Remember that DT sequences do not differentiate between a sequence and its mirror image. Thus, one can make a global change in sign of the $f(i)$ for all $i$, and still have a realizable sequence.

In [2]:
# Example sequence 3^l 5^u 7^- 1^+

seq = [[0, 3, -2], [2, 5, 2], [4, 7, -1], [6, 1, 1]]

orientList = isRealizable(seq, f_list = True)
orientList

[1, 1, -1, -1, 1, 1, -1, -1]

## Find the planar diagram code

The function `planarDiagram(seq, f_list)` takes the DT sequence `seq` and the orientation list `f_list`, and produces a planar diagram list from them. Thus, the function returns a list of two lists. The first list gives the PD code for the graph. **Note:** the numbers in this list refer to *half-edges* (or *darts*), not *nodes*. There can be problems if an induced knot diagram has self-loops, so using half-edges allows for a unique number associated with each of the four half-edges incident to the node.

A vertex is treated as the appropriate crossing, based on its vertex state. The second list gives the node type for each node. Again, we use 1 for a crossing, and 2 for a vertex. The sign of the numbers comes from $f(i)$, so they are positive if the upper crossing is right-to-left, and negative if left-to-right.

In [3]:
from Graphpoly.planarDiagram import planarDiagram

Here, we show the planar diagram notation for $3^\ell 5^u 7^- 1^+$, where the orientation is that found above in a previous cell. The node type, along with the value of $f(i)$, is recorded in `nodeTypeList`.

In [17]:
[PD_code, nodeTypeList] = planarDiagram(seq, orientList)

print('PD code =', PD_code)
print('node type list =', nodeTypeList)

PD code = [[0, 6, 1, 7], [2, 12, 3, 13], [8, 14, 9, 15], [10, 4, 11, 5]]
node type list = [2, 1, 1, 2]


## List all induced knot diagrams

We use a modified version of Hierholzer's algorithm to find all possible Eulerian circuits through the graph. Note that there is only one possible way to pass along the circuit when coming into a crossing, but three possible ways for a vertex. Thus, the algorithm must take this freedom into account, using the list `nodeTypeList` obtained from `planarDiagram()`.

In [5]:
from Graphpoly.createCircuits import createCircuits

The function `createCircuits()` returns a list of `graph` objects, each of which represents a particular choice of Eulerian circuit through the original graph. The half-edges of the circuit can be returned using `graph.listEPath()`. The labels of the half-edges are the same as those used in `planarDiagram()`.

In [6]:
Ecircuits = createCircuits(PD_code, nodeTypeList)
graphList = [graph for graph in Ecircuits]

for graph in graphList:
    print(graph.listEPath())
    
print('Number of circuits:', len(graphList))

[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 6, 5, 10, 9, 8, 7]
[0, 7, 8, 9, 10, 5, 6, 1, 2, 3, 4, 11, 12, 13, 14, 15]
[0, 6, 5, 10, 9, 8, 7, 1, 2, 3, 4, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 4, 3, 2, 1, 7, 8, 9, 10, 5, 6]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 10, 9, 8, 7, 1, 2, 3, 4, 5, 6]
[0, 6, 5, 4, 3, 2, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 5, 6, 1, 2, 3, 4, 10, 9, 8, 7]
[0, 7, 8, 9, 10, 4, 3, 2, 1, 6, 5, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 10, 9, 8, 7, 6, 5, 11, 12, 13, 14, 15]
[0, 15, 14, 13, 12, 11, 5, 6, 7, 8, 9, 10, 4, 3, 2, 1]
Number of circuits: 12


## Finding the 2-variable polynomial

Now we can combine the induced knot diagrams just obtained with the original PD code for the knotted graph. This will allow us to figure out how the Eulerian circuit passes through each vertex, and therefore which of the three skein relations given above to use for that vertex. The list `nodeTypeList` above gives information on whether the node is a vertex or a crossing. We will only look at nodes with values $\pm 2$ in that list.